In [ ]:
# default_exp seal_helper

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# seal_helper

In [ ]:
# export
import tenseal.sealapi as seal
from typing import List

In [ ]:
# export
def print_vector(vec, print_size=4, prec=3):
    """Prints a vector with a given level of precision and print size"""
    slot_count = len(vec)
    print()
    if slot_count <= 2*print_size:
        print("    [", end="")
        for i in range(slot_count):
            print(" " + (f"%.{prec}f" % vec[i]) + ("," if (i != slot_count - 1) else " ]\n"), end="")
    else:
        print("    [", end="")
        for i in range(print_size):
            print(" " + (f"%.{prec}f" % vec[i]) + ",", end="")
        if len(vec) > 2*print_size:
            print(" ...,", end="")
        for i in range(slot_count - print_size, slot_count):
            print(" " + (f"%.{prec}f" % vec[i]) + ("," if (i != slot_count - 1) else " ]\n"), end="")
    print()
    
def print_ptx(ptx):
    result = encoder.decode_double(ptx)
    print_vector(result, 3, 7)
    
def print_ctx(ctx):
    ptx = seal.Plaintext()
    decryptor.decrypt(ctx, ptx)
    print_ptx(ptx)
        
def print_range_ptx(ptx, end=0, begin=0):
    r = range(begin,end)
    
    values = encoder.decode_double(ptx)
    for i in r:
        print(f"{i} : {values[i]}")
    
def print_range_ctx(ctx, end=0, begin=0):
    ptx = seal.Plaintext()
    decryptor.decrypt(ctx, ptx)
    
    print_range_ptx(ptx, end, begin)

In [ ]:
# export
def float_to_ctx(x, encoder: seal.CKKSEncoder, encryptor: seal.Encryptor):
    ptx = seal.Plaintext()
    if len(x) > 1:
        x = list(x)
    encoder.encode(x, scale, ptx)

    ctx = seal.Ciphertext()
    encryptor.encrypt(ptx, ctx)
    
    return ctx

In [ ]:
# export
def vrep(x, n):
    k = n // len(x)
    rest = n % len(x)
    output = x * k + x[:rest]
    return output

In [ ]:
# export
def create_seal_globals(globals: dict, poly_modulus_degree: int, moduli: List[int], PRECISION_BITS: int,
                       use_local=True, use_symmetric_key=False):
    """Creates SEAL context variables and populates the globals with it."""
    parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
    parms.set_poly_modulus_degree(poly_modulus_degree)
    parms.set_coeff_modulus(seal.CoeffModulus.Create(
        poly_modulus_degree, moduli))

    context = seal.SEALContext.Create(parms, True, seal.SEC_LEVEL_TYPE.TC128)

    keygen = seal.KeyGenerator(context)
    
    globals["parms"] = parms
    globals["context"] = context
    globals["scale"] = pow(2.0, PRECISION_BITS)
    
    globals["public_key"] = keygen.public_key()
    globals["secret_key"] = keygen.secret_key()
    if use_local:
        globals["relin_keys"] = keygen.relin_keys_local()
        globals["galois_keys"] = keygen.galois_keys_local()
    else:
        globals["relin_keys"] = keygen.relin_keys()
        globals["galois_keys"] = keygen.galois_keys()
    
    if use_symmetric_key:
        globals["encryptor"] = seal.Encryptor(context, globals["secret_key"])
    else:
        globals["encryptor"] = seal.Encryptor(context, globals["public_key"])
        
    globals["evaluator"] = seal.Evaluator(context)
    globals["decryptor"] = seal.Decryptor(context, globals["secret_key"])
    globals["encoder"] = seal.CKKSEncoder(context)
    
def append_globals_to_builtins(globals, builtins):
    """Appends the SEAL context variables to the builtins.
    
    This allows the following variables to be called from functions globally. Only use for testing purposes.
    """
    
    variables = ["public_key", "secret_key", "relin_keys", "galois_keys",
                 "encryptor", "evaluator", "decryptor", "encoder", "scale", "parms", "context"]
    
    for var in variables:
        setattr(builtins, var, globals[var])

In [ ]:
# export
from pathlib import Path

def save_seal_globals(globals, path:Path = Path("seal")):
    parms = globals["parms"]
    
    public_key = globals["public_key"]
    secret_key = globals["secret_key"]
    relin_keys = globals["relin_keys"]
    galois_keys = globals["galois_keys"]
    
    if not path.exists():
        path.mkdir()
        
    parms.save(str(path/"parms"))
    
    public_key.save(str(path/"public_key"))
    secret_key.save(str(path/"secret_key"))
    relin_keys.save(str(path/"relin_keys"))
    galois_keys.save(str(path/"galois_keys"))

def load_seal_globals(globals, path:Path = Path("seal"), load_pk:bool = False, load_sk:bool = False):
    """Loads and populates SEAL globals from saved files."""
    if not path.exists():
        raise FileNotFoundError("Path not found")
        
    parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
    parms.load(str(path/"parms"))
    
    context = seal.SEALContext.Create(parms, True, seal.SEC_LEVEL_TYPE.TC128)
    globals["context"] = context
    
    if load_pk:
        public_key = seal.PublicKey()
        public_key.load(context, str(path/"public_key"))
        globals["public_key"] = public_key
        globals["encryptor"] = seal.Encryptor(context, public_key)
        
    if load_sk:
        secret_key = seal.SecretKey()
        secret_key.load(context, str(path/"secret_key"))
        globals["secret_key"] = secret_key
        globals["decryptor"] = seal.Decryptor(context, secret_key)
    
    relin_keys = seal.RelinKeys()
    relin_keys.load(context, str(path/"relin_keys"))
    
    galois_keys = seal.GaloisKeys()
    galois_keys.load(context, str(path/"galois_keys"))
    
    globals["relin_keys"] = relin_keys
    globals["galois_keys"] = galois_keys

    globals["evaluator"] = seal.Evaluator(context)
    globals["encoder"] = seal.CKKSEncoder(context)

The examples bellow shows how one can play with SEAL.

First we initialize the SEAL context :

In [ ]:
poly_modulus_degree = 4096
moduli = [35,30,35]
PRECISION_BITS = 30

create_seal_globals(globals(), poly_modulus_degree, moduli, PRECISION_BITS)

We can also save those parameters.

In [ ]:
save_seal_globals(globals())

In order to load them later, or to send them to a third party which will do computation on our data.

In [ ]:
load_seal_globals(globals())

Now we can start using the SEAL context to encrypt data and perform arithmetic on it.

In [ ]:
# First we encode x in a Plaintext
x = [1,2,3]

ptx = seal.Plaintext()
encoder.encode(x, scale, ptx)

# Then we display it
print_ptx(ptx)

# Then we encrypt it
ctx = seal.Ciphertext()
encryptor.encrypt(ptx, ctx)

print_ctx(ctx)

evaluator.add_plain_inplace(ctx, ptx)
print_ctx(ctx)


    [ 1.0000000, 2.0000000, 3.0000000, ..., -0.0000000, -0.0000000, -0.0000000 ]


    [ 0.9999999, 1.9999997, 3.0000000, ..., 0.0000002, -0.0000001, 0.0000000 ]


    [ 1.9999999, 3.9999997, 6.0000000, ..., 0.0000002, -0.0000001, 0.0000000 ]

